 # 1. 환경설정

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer #토큰나이저+벡터화(정수 인덱스)
from sklearn.model_selection import train_test_split #데이터를  train/test분할
import matplotlib.pyplot as plt #시각화 도구
from tensorflow.keras.models import Sequential #학습 모델
from tensorflow.keras.layers import Dense, Dropout, Embedding,Dense, LSTM, BatchNormalization
from tensorflow.keras.utils import to_categorical #수치화된 벡터를 원핫 인코딩
from konlpy.tag import Okt
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re




In [6]:
import pandas as pd
import numpy as np
train_data = pd.read_table('ratings_train.txt')[['document','label']]
test_data = pd.read_table('ratings_test.txt')[['document','label']]
print('훈련용 리뷰 개수 :', len(train_data))
print('테스트용 리뷰 개수', len(train_data))

                                          

훈련용 리뷰 개수 : 150000
테스트용 리뷰 개수 150000


In [7]:
p_data=test_data

In [8]:
p_data

,document,label
0,굳 ㅋ,1
1,GDNTOPCLASSINTHECLUB,0
2,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...
49995,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


# 2. 텍스트 전처리
## 2-1. 정제&정규화

In [14]:
#1. 중복제거
p_data=p_data.drop_duplicates(subset=['document'])
#2. 한글, 공백 외 문자 제거
p_data['document']=p_data.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
#3. 여러 공백은 하나의 공백으로
p_data['document']=p_data.document.str.replace('^ +','')
#4. 하나의 공백은 NaN처리
p_data['document'] = p_data.document.replace('',np.nan)
#5. NaN가 있는 document값은 삭제
p_data= p_data.dropna(how='any')
s_w = ['은','는','이','가','를','에게','의','을','도','으로','만','라서','하다']


C:\Users\JSWonner\AppData\Local\Temp/ipykernel_7304/4023423763.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  p_data['document']=p_data.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
C:\Users\JSWonner\AppData\Local\Temp/ipykernel_7304/4023423763.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  p_data['document']=p_data.document.str.replace('^ +','')


In [16]:
type(p_data.document)

pandas.core.series.Series

okt = Okt()
from tqdm import tqdm
X_data = [] #토큰화된 문서
for sent in tqdm(p_data.document):
    tk_d = okt.morphs(sent)
    end_d = [w for w in tk_d if w not in s_w]
    X_data.append(end_d)
X_data

In [22]:
type(p_data.label)

pandas.core.series.Series

In [24]:
tk=Tokenizer() #케라스 토크나이자. 이미 토큰화가 완료된 문서
tk.fit_on_texts(X_data) #단어 사전 생성
y_train = np.array(p_data.label)

In [25]:
X_train= tk.texts_to_sequences(X_data) #단어 인덱스로 인코딩(벡터화)

In [26]:
#인코딩된 문장에 단어가 하나도 없다면 그 행을 추출
drop_train=[index for index,sent in enumerate(X_train) if len(sent)<1]

In [30]:
#의미가 없는 공백 행 삭제
X_train = np.delete(X_train,drop_train,axis=0) #높은 차원 축
y_train = np.delete(y_train,drop_train,axis=0)